In [2]:
import pandas as pd

In [3]:
from pandas import Series, DataFrame

## 데이터셋 불러오기

In [4]:
keywords = pd.read_csv('./dataset/금지어keyword.csv')
blacklist = pd.read_csv('./dataset/블랙리스트정보v2 - 복사본.csv', encoding='latin1')
normal_item_2022 = pd.read_csv('./dataset/정상상품.csv')
weird_item_2022 = pd.read_csv('./dataset/이상상품.csv')
normal_member = pd.read_csv('./dataset/정상회원데이터.csv')
weird_member = pd.read_csv('./dataset/이상회원데이터.csv')
inquir = pd.read_csv('./dataset/인콰이어리데이터.csv')

C:\Users\ujn9\AppData\Local\Temp\ipykernel_4044\3917221033.py:5: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  normal_member = pd.read_csv('./dataset/정상회원데이터.csv')


In [4]:
members = pd.read_csv('./dataset/members_concat_v2.csv')
items = pd.read_csv('./dataset/product_concat_v2.csv')

In [4]:
items.head()

,MEMBER_ID,CATALOG_NM,CATEGORYM_ID,KEYWORD,DISPLAY,CATALOG_DESC,INPUT_DT,UPDATE_DT,REMOTEIP,SHOW_CHK,judge
0,shangyuanshibang,1-BOC-4-(4-BROMO-PHENYLAMINO)-PIPERIDINECas 44...,212815,1-BOC-4- 4-BROMO-PHENYLAM,1,1. 名称和标识符1.1 名称1-叔丁氧羰基-4-（4-溴苯基氨基）哌啶1.2 同义词1-哌...,20230228,20230228,21978227175,1,0
1,shangyuanshibang,High Purity N- (Tert-Butoxycarbonyl) -4-Piperi...,212815,N- tert-Butoxycarbonyl -4,1,1. 名称和标识符1.1 名称N-叔丁氧羰基-4-哌啶酮1.2 同义词1-叔丁氧羰基-4-哌...,20230228,20230228,4237992,1,0
2,shangyuanshibang,"1,1-dimethylethyl Ester CAS 288573-56-8",212815,1-BOC-4- 4-FLUORO-PHENYLA,1,1. Names and Identifiers1.1 Name1-BOC-4-(4-FLU...,20230228,20230228,4237992,1,0
3,shangyuanshibang,White Crystal N-Isopropylbenzylamine CAS 102-9...,212815,N-Isopropylbenzylamine,1,1. Names and Identifiers1.1 NameN-Isopropylben...,20230228,20230228,4237992,1,0
4,5527,Soybean GMO & NON-GMO,11607,Soybean soybean gmo,1,Soja GMO & Soja NON-GMOEm estoque: 5000000 MT...,20230228,20230228,131196216214,1,0


## 패키지 로드

In [5]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
import string

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ujn9\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## 데이터 클렌징

### 데이터 드롭

In [19]:
normal_df = normal_df.drop('MEMBER_ID', axis=1)
weird_df = weird_df.drop('MEMBER_ID', axis=1)

In [6]:
members = members.drop('MEMBER_ID', axis=1)
items = items.drop('MEMBER_ID', axis=1)

In [7]:
items.isna().sum()

CATALOG_NM      0
CATEGORYM_ID    0
KEYWORD         0
DISPLAY         0
CATALOG_DESC    0
INPUT_DT        0
UPDATE_DT       0
REMOTEIP        0
SHOW_CHK        0
judge           0
dtype: int64

### NA처리

In [22]:
normal_df.fillna(0, inplace=True)

# Fill NaN values with 0 in weird_df in-place
weird_df.fillna(0, inplace=True)

In [23]:
normal_df.head()

,GCATALOG_ID,CATALOG_ID,CATALOG_NM,CATEGORYM_ID,KEYWORD,DISPLAY,CATALOG_DESC,INPUT_DT,UPDATE_DT,REMOTEIP,SHOW_CHK,judge
0,GC11346343,CA11736663,New Season 2023 Black Tea Ctc for Milk Tea Fro...,1538,"Milk tea,Ctc black tea,Fulmex tea",0,Black tea ctcgrade: pfpacking 65kg/bagqt 26 to...,20230228005732,20230228005732,14.177.113.144,0,1
3,GC11725829,CA11736659,6mm 612 Electric DC Mini Vibrator Motor VR Mas...,1729,"vibration motor,dc motor,coreless motor,mini m...",0,6mm 612 Electric DC Mini Vibrator Motor VR Mas...,20230227223506,20230227223506,103.167.134.61,0,1
5,GC11736640,CA11736657,Nintendo Switch Console OLED Model 64GB White ...,84599,Nintendo Switch,0,+1 (203) 684 0957.WHATSAPP +1 203 684 0957Stoc...,20230227204731,20230227204731,154.6.85.152,0,1
6,GC11736640,CA11736656,Newest Roku Ultra LT Streaming Device 4K/HDR/D...,381502,Roku,0,+1 (203) 684 0957.WHATSAPP +1 203 684 0957Stoc...,20230227204555,20230227204555,154.6.85.152,0,1
7,GC11736640,CA11736655,SAME DAY SHIP! OlaplexING NO. 7 Bonding Oil- 1...,310902,OlaplexING,0,+1 (203) 684 0957.WHATSAPP +1 203 684 0957Stoc...,20230227204422,20230227204422,154.6.85.152,0,1


### lower(), 불용어, 어근추출

In [9]:
def preprocess_text(text):
    if isinstance(text, str):  # Check if the value is a string
        # Convert to lowercase
        text = text.lower()

        # Remove special characters
        text = ''.join([char for char in text if char not in string.punctuation])

    return text

def remove_stopwords(text):
    if isinstance(text, str):  # Check if the value is a string
        stop_words = set(stopwords.words('english'))
        words = text.split()
        filtered_words = [word for word in words if word.lower() not in stop_words]
        return ' '.join(filtered_words)
    return text

def apply_stemming(text):
    if isinstance(text, str):  # Check if the value is a string
        stemmer = PorterStemmer()
        words = text.split()
        stemmed_words = [stemmer.stem(word) for word in words]
        return ' '.join(stemmed_words)
    return text

# Apply text processing to specified columns
columns_to_process = ['CATALOG_DESC', 'CATALOG_NM', 'KEYWORD']
for column in columns_to_process:
    items[column] = items[column].apply(preprocess_text)
    items[column] = items[column].apply(remove_stopwords)
    items[column] = items[column].apply(apply_stemming)
  

### 중국말 삭제

In [42]:
import re

In [43]:
items['CATALOG_DESC'] = items['CATALOG_DESC'].apply(lambda x: re.sub('[^a-zA-Z]',' ',str(x)))

In [44]:
items['CATALOG_NM'] = items['CATALOG_NM'].apply(lambda x: re.sub('[^a-zA-Z]',' ',str(x)))

In [45]:
items['KEYWORD'] = items['KEYWORD'].apply(lambda x: re.sub('[^a-zA-Z]',' ',str(x)))

In [61]:
items.head()

,CATALOG_NM,CATEGORYM_ID,KEYWORD,DISPLAY,CATALOG_DESC,INPUT_DT,UPDATE_DT,REMOTEIP,SHOW_CHK,judge
0,boc bromophenylaminopiperidineca,212815,boc bromophenylam,1,...,20230228,20230228,21978227175,1,0
1,high puriti n tertbutoxycarbonyl piperidon ca...,212815,n tertbutoxycarbonyl,1,n ...,20230228,20230228,4237992,1,0
2,dimethylethyl ester ca,212815,boc fluorophenyla,1,name identifiers name boc fluorophenylami...,20230228,20230228,4237992,1,0
3,white crystal nisopropylbenzylamin ca i...,212815,nisopropylbenzylamin,1,name identifiers namenisopropylbenzylamine...,20230228,20230228,4237992,1,0
4,soybean gmo nongmo,11607,soybean soybean gmo,1,soja gmo soja nongmoem estoqu mtorigem...,20230228,20230228,131196216214,1,0


In [64]:
items['CATALOG_DESC']

0                                                       ...
1                      n                                ...
2           name identifiers   name boc  fluorophenylami...
3           name identifiers   namenisopropylbenzylamine...
4         soja gmo soja nongmoem estoqu         mtorigem...
                                ...                        
751310    product name serilesin synonym hexapeptide   p...
751311    product descript hrc   blue nano coat mill too...
751312    product descript hrc   blue nano coat mill too...
751313    product descript hrc   blue nano coat mill too...
751314    product descript hrc   blue nano coat mill too...
Name: CATALOG_DESC, Length: 751315, dtype: object

In [46]:
x = items.drop('judge', axis=1)


In [47]:
y = items['judge']

In [48]:
t = x.head(5000)

In [49]:
q = y.head(5000)

## 데이터 분리

In [17]:
from sklearn.model_selection import train_test_split

### object 타입 원핫인코딩

In [ ]:
t = pd.get_dummies(t)

## 모델 검증 절차

###  다중공선성 확인

In [49]:
t.columns

Index(['CATEGORYM_ID', 'DISPLAY', 'INPUT_DT', 'UPDATE_DT', 'SHOW_CHK',
       'REMOTEIP_no_period',
       'CATALOG_NM_0130mm carbon steel coil hot roll astm a283a283m03',
       'CATALOG_NM_018mm 304 stainless steel strip spring coil',
       'CATALOG_NM_018mm adhes insul tape electr 180c silicon adhes tape',
       'CATALOG_NM_01mm 02mm 1145 1035 h16 aluminum coilfoil light protect',
       ...
       'CATALOG_DESC_通用名2甲基3苯基环氧乙烷2甲酸cas号5449127分子量20117400密度128gcm3沸点3343ºc 760 mmhg分子式c10h10nao3熔点na闪点1367ºc精确质量20105300psa49130600logpindex2of1折射1575包装及发货 包装规格： 1可定制 2现有包装 1kg包装：铝袋包装，内有自封袋。10kg纸箱（三高：500mm、300mm、400mm）25kg桶装（直径：350mm高：510mm）纸桶包装，内衬双层塑料袋。下订单最小起订量：10 克包装：铝箔袋定制桶付款条件：西联汇款货币克btcttlc 运输方式：空运或海运快递：fedex、ups、dhl、tnt ，em 装货港：中国上海可选交货提前期：3 天内运输：我们通过 dhl、fedex、ups、tnt、中国邮政、nl 邮政和其他快递公司运送货物，重量从 10 克到 1000 公斤甚至散货船。运输细节和运输文件将在准备就绪后提供并发送给客户。我们会一直跟踪发货情况，直到客户收到满意为止。经过多年的出口海运经验，与专业、经验丰富的货代合作，确保货物以多种方式安全、高效地送达。',
       'CATALOG_DESC_通用名3氧代4苯基丁酸乙酯cas号5413058分子量206238密度11±01 gcm3沸点2903±150 °c 7

In [101]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Assuming X is your independent variable matrix
vif_data = pd.DataFrame()
vif_data["Variable"] = merged_feature.columns
vif_data["VIF"] = [variance_inflation_factor(merged_feature.values, i) for i in range(merged_feature.shape[1])]

print(vif_data)

             Variable       VIF
0        CATEGORYM_ID  1.003994
1             DISPLAY  1.017272
2            INPUT_DT  0.000004
3           UPDATE_DT  1.027287
4            SHOW_CHK  1.011617
5  REMOTEIP_no_period  1.001096


### 샘플데이터 생성

In [24]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [25]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=0)

# Decision Tree Model
decision_tree_model = DecisionTreeClassifier(random_state=22)
decision_tree_model.fit(X_train, y_train)

# Random Forest Model
random_forest_model = RandomForestClassifier(random_state=22)
random_forest_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred_decision_tree = decision_tree_model.predict(X_test)
y_pred_random_forest = random_forest_model.predict(X_test)

# Evaluate Decision Tree Model
print("Decision Tree Model:")
print("Accuracy:", accuracy_score(y_test, y_pred_decision_tree))
print("Classification Report:\n", classification_report(y_test, y_pred_decision_tree))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_decision_tree))

# Evaluate Random Forest Model
print("\nRandom Forest Model:")
print("Accuracy:", accuracy_score(y_test, y_pred_random_forest))
print("Classification Report:\n", classification_report(y_test, y_pred_random_forest))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_random_forest))

ValueError: Found input variables with inconsistent numbers of samples: [9, 751315]

### 과적합 확인

In [103]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(random_forest_model, X_train, y_train, cv=5)  # 5-fold cross-validation
print("Random_forest_model Cross-Validation 평균 정확도:", scores.mean())

Random_forest_model Cross-Validation 평균 정확도: 0.9998558756333471


In [104]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(decision_tree_model, X_train, y_train, cv=5)  # 5-fold cross-validation
print("Decision_tree_model Cross-Validation 평균 정확도:", scores.mean())

Decision_tree_model Cross-Validation 평균 정확도: 0.9997866961666748


# TF-IDF

In [19]:
from scipy.sparse import csr_matrix

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [58]:
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 1))
tfidf_matrix = tfidf_vectorizer.fit_transform(t['CATALOG_DESC'])

# Convert sparse matrix to dense array
dense_array = tfidf_matrix.toarray()

# Create new DataFrame with the dense array
tfidf_df = pd.DataFrame(dense_array, columns=tfidf_vectorizer.get_feature_names_out())

# Replace the 'CATALOG_DESC' column in the original DataFrame with the new DataFrame
tt = pd.concat([t.drop('CATALOG_DESC', axis=1), tfidf_df], axis=1)

In [ ]:
# TF-IDF vectorization
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(t)

# Convert the sparse matrix to CSR format
tfidf_matrix_csr = csr_matrix(tfidf_matrix)

# Create Isolation Forest model
isolation_forest = IsolationForest(contamination=0.5, random_state=42)

# Model training
isolation_forest.fit(tfidf_matrix_csr)  # Use the converted tfidf_matrix_csr here

# Calculate outlier scores
outlier_scores = isolation_forest.decision_function(tfidf_matrix_csr)

# Add outlier scores to data frame
product_df['outlier_score'] = outlier_scores

# Set the threshold to judge as an outlier (e.g., consider the top 5% of samples as outliers)
threshold = product_df['outlier_score'].quantile(0.95)

# Add a column indicating whether there is an outlier or not
product_df['is_outlier'] = product_df['outlier_score'] < threshold

# Output results
print(product_df[['Product Name', 'outlier_score', 'is_outlier']])

NameError: name 'IsolationForest' is not defined

## 그래디언트-부스팅

In [20]:
pip install xgboost

  Obtaining dependency information for xgboost from https://files.pythonhosted.org/packages/bc/43/242432efc3f60052a4a534dc4926b21e236ab4ec8d4920c593da3f65c65d/xgboost-2.0.2-py3-none-win_amd64.whl.metadata
   ---------------------------------------- 0.0/99.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/99.8 MB 991.0 kB/s eta 0:01:41
   ---------------------------------------- 0.6/99.8 MB 7.1 MB/s eta 0:00:14
   ---------------------------------------- 1.1/99.8 MB 8.7 MB/s eta 0:00:12
    --------------------------------------- 1.7/99.8 MB 9.6 MB/s eta 0:00:11
    --------------------------------------- 2.2/99.8 MB 10.1 MB/s eta 0:00:10
   - -------------------------------------- 2.8/99.8 MB 10.4 MB/s eta 0:00:10
   - -------------------------------------- 3.3/99.8 MB 10.7 MB/s eta 0:00:10
   - -------------------------------------- 3.9/99.8 MB 10.8 MB/s eta 0:00:09
   - -------------------------------------- 4.4/99.8 MB 10.9 MB/s eta 0:00:09
   -- -------------------


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: C:\Users\ujn9\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [31]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.datasets import load_iris

In [37]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# Create a XGBoost Classifier
xgb_classifier = xgb.XGBClassifier(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)

# Fit the model to the training data
xgb_classifier.fit(X_train, y_train)

# Make predictions on the test set
predictions = xgb_classifier.predict(X_test)

# Evaluate the accuracy of the model
accuracy = accuracy_score(y_test, predictions)
print(f"Accuracy: {accuracy}")

ValueError: Found input variables with inconsistent numbers of samples: [9, 751315]

In [ ]:
import xgboost as xgb
from xgboost import XGBClassifier

# Extreme Gradient Boosting (xgboost) 모델 사용

# 1. 모델 선언
xgb  = XGBClassifier()

# 2. 모델 훈련 fit()함수
xgb.fit(X_train, Y_train)

# 3. 모델 예측 predict()함수
Y_pred = xgb.predict(X_test)

# 4. score()
xgb.score(X_train, Y_train)

In [28]:
    from sklearn.model_selection import cross_val_score

    scores = cross_val_score(xgb_classifier, X_train, y_train, cv=5)  # 5-fold cross-validation
    print("Random_forest_model Cross-Validation 평균 정확도:", scores.mean())

Random_forest_model Cross-Validation 평균 정확도: 1.0
